### Kaggle Data Science BOWL 2017 
collect cancer data from LUNA and non-cancer from DSB

In [48]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
#import seaborn as sns
import os
import glob
import cv2
import scipy.ndimage as ndimage
from sklearn.cross_validation import KFold
from skimage import measure, morphology, segmentation
#from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import re
import traceback
import dicom
%matplotlib inline
#p = sns.color_palette()
import time
from sklearn import cross_validation

# get package versions
def get_version(*vars):
    for var in vars:
        module = __import__(var)    
        print '%s: %s' %(var,module.__version__)
    
# package version    
get_version('numpy','matplotlib','cv2','sklearn','skimage','scipy')

numpy: 1.11.2
matplotlib: 1.5.1
cv2: 3.1.0
sklearn: 0.18.1
skimage: 0.12.3
scipy: 0.18.1


### presets

In [6]:
path2dsb='./output/data/dsb/'
path2luna='./output/data/luna/'
path2csv='./output/data/stage1_labels.csv'
path2output='./output/data/'

# dispaly
display_ena=False

# resize
H,W=512,512

### utils

In [35]:
def array_stats(X):
    X=np.asarray(X)
    
    # get var name
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    
    print (vars_name,X.shape, X.dtype)
    #print ('array shape',X.shape, X.dtype)
    #print 'min: %.3f, max:%.3f, avg: %.3f, std:%.3f' %(np.min(X),np.max(X),np.mean(X),np.std(X))
    print ('min: {}, max: {}, avg: {:.3}, std:{:.3}'.format( np.min(X),np.max(X),np.mean(X),np.std(X)))
    
    print '-'*50
    

    
    

# stack images and labels
def stack_data(df,indices,(h,w)):
    X=[]
    Y=[]
    y=[]
    for r in (indices):
        # get patinet id and cancer status
        p_id=df_train.id[r] # patient id
        p_c=df_train.cancer[r] # patinet cancer

        # load scan
        scan=load_scan(path2data+p_id)
        X1 = get_pixels_hu(scan)
        nb_slices=len(X1)
        N,H,W=X1.shape
        
        # get lung segmentation
        #print 'wait ....'
        Y1=np.zeros_like(X,'uint8')
        for slice_nm in range(len(X1)):
            masks=seperate_lungs(X[slice_nm])
            Y1[slice_nm]=masks[1]
        
        # resize
        if h<H:
            Xr=np.zeros((nb_slices,h,w),dtype=X1.dtype)
            Yr=np.zeros((nb_slices,h,w),dtype=Y1.dtype)
            for k1 in range(nb_slices):
                Xr[k1]=cv2.resize(X1[k1], (w, h), interpolation=cv2.INTER_CUBIC)
                Yr[k1]=cv2.resize(Y1[k1], (w, h), interpolation=cv2.INTER_CUBIC)
        else:
            Xr=X1
            Yr=Y1
        print 'patient %s id: %s, cancer: %s, nb slices: %s' %(r,p_id,p_c,Xr.shape[0])        
        X.append(Xr)
        Y.append(Yr)
        y.append(p_c)
    return X,y  

def grays_to_RGB(img):
    # turn 2D grayscale image into grayscale RGB
    return np.dstack((img, img, img))



def image_with_mask(img, mask,color=(0,255,0)):
    maximg=np.max(img)    
    img=np.asarray(img,dtype='float32')
    img=np.asarray((img/maximg)*255,dtype='uint8')
    mask=np.asarray(mask,dtype='uint8') 
    if np.max(mask)==1:
        mask=mask*255

    # returns a copy of the image with edges of the mask added in red
    if len(img.shape)==2:	
	img_color = grays_to_RGB(img)
    else:
	img_color =img

    mask_edges = cv2.Canny(mask, 100, 200) > 0
    img_color[mask_edges, 0] = color[0]  # set channel 0 to bright red, green & blue channels to 0
    img_color[mask_edges, 1] = color[1]
    img_color[mask_edges, 2] = color[2]
    img_color=img_color#/float(np.max(img))
    return img_color

def disp_img_2masks(img,mask1,mask2,r=1,c=1,d=0,indices=None):
    if mask1 is None:
        mask1=np.zeros(img.shape,dtype='uint8')
    if mask2 is None:
        mask2=np.zeros(img.shape,dtype='uint8')
        
    N=r*c    
    if d==2:
        # convert to N*C*H*W
        img=np.transpose(img,(2,0,1))
        img=np.expand_dims(img,axis=1)
        
        mask1=np.transpose(mask1,(2,0,1))
        mask1=np.expand_dims(mask1,axis=1)

        mask2=np.transpose(mask2,(2,0,1))
        mask2=np.expand_dims(mask2,axis=1)
        
    if indices is None:    
        # random indices   
        n1=np.random.randint(img.shape[0],size=N)
    else:
        n1=indices
    
    I1=img[n1,0]
    #M1=mask1[n1,0]
    M1=np.zeros(I1.shape,dtype='uint8')
    for c1 in range(mask1.shape[1]):
        M1=np.logical_or(M1,mask1[n1,c1,:])    
    #M2=mask2[n1,0]
    M2=np.zeros(I1.shape,dtype='uint8')
    for c1 in range(mask2.shape[1]):
        M2=np.logical_or(M2,mask2[n1,c1,:])    
    
    C1=(0,255,9)
    C2=(255,0,0)
    for k in range(N):    
        imgmask=image_with_mask(I1[k],M1[k],C1)
        imgmask=image_with_mask(imgmask,M2[k],C2)
        plt.subplot(r,c,k+1)
        plt.imshow(imgmask)
        plt.title(n1[k])
    plt.show()            
    #return n1        


### DSB Training stats

In [19]:
df_train = pd.read_csv(path2csv)
print('Number of training patients: {}'.format(len(df_train)))
print('Cancer rate: {:.4}%'.format(df_train.cancer.mean()*100))
df_train.head()

# extract non cancers
non_cancer=df_train[df_train.cancer==0].index
cancer=df_train[df_train.cancer==1].index

print 'total non cancer:%s, total cancer:%s' %(len(non_cancer),len(cancer))


Number of training patients: 1397
Cancer rate: 25.91%
total non cancer:1035, total cancer:362


#### sample data

In [42]:
k=np.random.randint(len(non_cancer))
p_id=df_train.id[non_cancer[k]]
p_c=df_train.cancer[non_cancer[k]]
print 'subject id: %s, cancer: %s' %(p_id,p_c)

# load data
f1=np.load(path2dsb+p_id+'.npz')
X=f1['X']
y=f1['y']
array_stats(X)

subject id: 862d1d80391d146b2cd02478a798b365, cancer: 0
(u'X', (103, 512, 512), dtype('int16'))
min: -1024, max: 1863, avg: -5.07e+02, std:4.73e+02
--------------------------------------------------


## collect non-cancer from DSB

In [56]:
start_time=time.time()

# total subjects
nb_sbj=len(non_cancer)
y=np.zeros(nb_sbj,dtype='uint8')

# train val split
trn_nc, val_nc, trn_y, val_y = cross_validation.train_test_split(non_cancer,y, random_state=420, stratify=y,
test_size=0.1)

import random

bs=8
X=[]
# pick bs random subjects
rnd_nc_inds=random.sample(trn_nc,bs)    

X=np.zeros((bs,3,H,W),'int16')
for k,ind in enumerate(rnd_nc_inds):
    p_id=df_train.id[ind]
    print p_id

    # load data
    f1=np.load(path2dsb+p_id+'.npz')
    X0=f1['X']

    # pick three concecutive slices
    n1=np.random.randint(X0.shape[0])
    #print n1
    X0=X0[n1:n1+3]
    #array_stats(X0) 
    X[k,:]=X0

array_stats(X)     
elapsed_time=(time.time()-start_time)
print ('elapsed time: %.2d sec' %(elapsed_time))    

234ac5dd589d09a4b2a753ff206d5588
afc15e047f3e127871d13e39cde7557d
1183f213c1c821df18aad63890920403
f79681d8561b6443a5b95f78f3ea6411
2b55d9c3f8e05951c87e90d2361aca6a
20809d879e5a966cc537beb42735a67f
ee9c580272cd02741df7299892602ac7
f88ed45f3436cb30e494409dae76df5d
(u'X', (8, 3, 512, 512), dtype('int16'))
min: -2048, max: 12116, avg: -4.82e+02, std:5.52e+02
--------------------------------------------------
elapsed time: 06 sec


## LUNA data set

In [81]:
from glob import glob
patients=glob(path2luna+'images*')
patients.sort()
print 'total subjects: %s' %(len(patients))

X=np.zeros((len(patients),3,H,W),dtype='int16')
for k,p in enumerate(patients):
    print k,p
    X0=(np.load(p)).astype('int16')
    X[k,:]=X0

# save
print 'wait'
array_stats(X)
np.save(path2luna+'luna.npy',X)
print 'luna saved'

total subjects: 1186
0 ./output/data/luna/images_0000_0537.npy
1 ./output/data/luna/images_0000_0600.npy
2 ./output/data/luna/images_0000_0601.npy
3 ./output/data/luna/images_0000_0606.npy
4 ./output/data/luna/images_0000_0607.npy
5 ./output/data/luna/images_0000_0651.npy
6 ./output/data/luna/images_0000_0652.npy
7 ./output/data/luna/images_0000_0732.npy
8 ./output/data/luna/images_0000_0733.npy
9 ./output/data/luna/images_0001_0000.npy
10 ./output/data/luna/images_0001_0001.npy
11 ./output/data/luna/images_0001_0003.npy
12 ./output/data/luna/images_0001_0004.npy
13 ./output/data/luna/images_0001_0005.npy
14 ./output/data/luna/images_0001_0006.npy
15 ./output/data/luna/images_0001_0008.npy
16 ./output/data/luna/images_0001_0010.npy
17 ./output/data/luna/images_0001_0014.npy
18 ./output/data/luna/images_0002_0002.npy
19 ./output/data/luna/images_0002_0007.npy
20 ./output/data/luna/images_0002_0009.npy
21 ./output/data/luna/images_0002_0012.npy
22 ./output/data/luna/images_0002_0013.npy
